# Instrumentacion y control: Practica 1 - Placa de audio #

## Introducción ## 
En este notebook vamos a implementar y registrar mediciones utilizando una placa de sonido. 

Para eso utilizamos el paquete soundDevice y desarrollamos codigo propio de manera de adaptar las funcionalidades que ofrece dicho paquete. El objetivo es tener montado sobre las funcionalidades de python y soundDevice una API que permita usar en forma rapida e intuitiva la placa de sonido para interactuar con dispotivos electronicos, usando el speaker como salida y el mic como entrada.

## Funcionalidades buscadas ##

- Generar señales periodicas tipicas (sinusoidal, triangular, cuadrada, etc)
- Registrar respuestas del sistema conectado frente a estimulos.
- Testear y autocaracterizar el sistema de la placa de sonido (ya que puede cambiar de computadora en computadora) 
- Conocer desfazaje entre estimulo enviado y respuesta detectada. 

## Materiales y Diseño experimental basico ## 

Para hacer funcionar el sistema se necesita una computadora o dispositivo que cuente con una salida de audio y una entrada fisicamente independiente de microfono. Ademas se necesita dos conectores Jack Mini Plug de 3.5mm (o la que corresponda al hardware donde se ejecute el codigo) stereo que terminen en cables sueltos. En nuestro caso para realizar todas las prueba experimentales conectamos dicho terminales a un protoboard. 

![setup inicial](Files/Imagenes/Setup1.jpeg)
![setup inicial](Files/Imagenes/Setup2.jpeg)

## Desarrollo de sofware ##

El codigo desarrollado esta incluido en el archivo AudioControl.py en forma de funciones de manera tal que se pueda importar el archivo y utilizarlas en forma directa. 
Describimos brevemente las funciones utiles:

